## 1. Import Libraries and Load Dataset File after Preprocessing and Labelling

In [23]:
"""
Meng-import library yang akan dibutuhkan untuk melakukan proses
pemodelan dengan 2 Algoritma Machine Learning: Naive Bayes dan Support Vector Machine (SVM)
"""
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB # Library untuk model naive bayes yang dikhususkan untuk memproses data teks
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

In [24]:
"""
Membaca file dataset setelah dilakukan proses pelabelan dengan 3 nilai label sentimen (positif, negatif dan netral)
dan juga telah dilakukan text processing untuk seluruh data komentar yang berhasil diambil dari video debat
babak pertama hingga kelima capres - cawapres tahun 2024 yang di-upload/ditayangkan melalui 3 channel YouTube yakni
Najwa Shihab, KOMPASTV dan KPU RI
"""
youtube_comments_df = pd.read_excel("Dataset/Text Processing and Labeling Results Dataset.xlsx")

youtube_comments_df.head(9) # Menampilkan 9 data teratas dari variabel DataFrame "youtube_comments_df"

,video_id,channel_id,channel_name,comment_id,author,dirty_text_comment,clean_text_comment,like_count,total_reply,published_date,updated_date,subjectivity,polarity,sentiment_label
0,Sbjsg3AFx00,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,Ugx6mhUfDOUhd65fWyh4AaABAg,@udinlamidun9716,Siapa yg setuju 3 capres di undang bertiga di ...,tuju capres undang tiga mata najwa debat nyata...,3970,332,2024-01-07 17:03:53,2024-01-09 23:18:49,0.300000,0.200000,positif
1,gUz_MgdwKg0,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,UgxfMHgX25B2b3jkO4B4AaABAg,@zakishofi3442,"Saya bukan pendukung siapapun, tapi terimakasi...",dukung terimakasih anies bicara kanjuruhan hre...,3182,91,2023-12-12 21:22:08,2023-12-12 21:22:08,0.000000,0.000000,netral
2,tEbRB5xxOPI,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,UgwTpSK1hjkTNQlcT894AaABAg,@sandigustiana,"Timestamp<br><br><a href=""https://www.youtube....",timestamp href https www youtube com watch teb...,2936,124,2024-01-21 21:32:13,2024-01-21 21:32:13,0.000000,0.000000,netral
3,tEbRB5xxOPI,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,UgzaWyuIp8PPLQrUOct4AaABAg,@abizardfahry4591,Siapa yang disini setuju 3 capres dan cawapres...,tuju capres cawapres undang tiga mata najwa de...,2531,91,2024-01-22 00:17:18,2024-01-22 00:17:18,0.300000,0.200000,positif
4,KJdt-HBBGIo,UCVdSz86o9q2cQKvAQ5QhOYg,KPU RI YouTube Channel,Ugy7bfVbuDxBgae9eVp4AaABAg,@nabilbudiman271,"<a href=""https://www.youtube.com/watch?v=KJdt-...",href https www youtube com watch kjdt hbbgio a...,1590,122,2024-01-07 15:37:05,2024-01-07 15:37:05,0.474444,0.313333,positif
5,gUz_MgdwKg0,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,UgzKC1PcDJZk7TvQ_Qh4AaABAg,@UmmuAzizah2010,Maa syaa Allaah Pak Anies ❤❤❤❤ <br>Terima kasi...,maa syaa allaah anies terima kasih cerah sbg r...,1542,23,2023-12-12 16:27:52,2023-12-12 16:27:52,0.800000,0.700000,positif
6,gUz_MgdwKg0,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,Ugxtzzc3zKJ8nY3Sg7F4AaABAg,@nursusilo7298,Tak terbantahkan Pak Anis cards yg paling siap...,bantah anis cards logis sistematis educativ ko...,1501,91,2023-12-12 21:00:05,2023-12-12 21:00:05,1.000000,0.900000,positif
7,JFl2fOTBNIE,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,UgwB34PptWoq1w3z7xN4AaABAg,@lilissuryanilimbong7685,Selaku Ibu Rumah Tangga saya tahu apa yg dibut...,rumah tangga butuh anak makan gratis solusi ma...,1486,169,2024-02-04 20:29:41,2024-02-04 20:29:41,0.703611,0.368333,positif
8,gUz_MgdwKg0,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,Ugypp7MrvLOhoz9GGUV4AaABAg,@abisopiyan8628,Saya dan keluarga berkat debat pertama ini ras...,keluarga berkat debat pilih anies baswedan pre...,1452,54,2023-12-12 17:49:04,2023-12-12 17:49:04,0.200000,0.200000,positif


## 2. Exploratory Dataset after Finished Labelling for Sentiment and Text Processing

In [25]:
# Menampilkan hanya nilai yang ada di dalam masing-masing kolom tetapi kolom tidak diikutsertakan
youtube_comments_df.values

array([['Sbjsg3AFx00', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.3, 0.2, 'positif'],
       ['gUz_MgdwKg0', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.0, 0.0, 'netral'],
       ['tEbRB5xxOPI', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.0, 0.0, 'netral'],
       ...,
       ['gUz_MgdwKg0', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.9, -0.8, 'negatif'],
       ['gUz_MgdwKg0', 'UCo8h2TY_uBkAVUIc14m_KCA',
        'Najwa Shihab YouTube Channel', ..., 0.0, 0.0, 'netral'],
       ['8J66JxvmEzo', 'UCVdSz86o9q2cQKvAQ5QhOYg',
        'KPU RI YouTube Channel', ..., 0.0, 0.0, 'netral']], dtype=object)

In [29]:
# Menampilkan jumlah baris dan kolom yang saat ini dimiliki oleh variabel DataFrame "youtube_comments_df"
youtube_comments_df.shape

(48239, 14)

In [30]:
# Menampilkan nama-nama kolom yang dimiliki oleh variabel DataFrame "youtube_comments_df"
youtube_comments_df.columns

Index(['video_id', 'channel_id', 'channel_name', 'comment_id', 'author',
       'dirty_text_comment', 'clean_text_comment', 'like_count', 'total_reply',
       'published_date', 'updated_date', 'subjectivity', 'polarity',
       'sentiment_label'],
      dtype='object')

Saat ini variabel DataFrame "youtube_comments_df" memiliki 14 kolom diantaranya adalah kolom "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment", "like_count", "total_reply", "published_date", "updated_date", "subjectivity", "polarity", "sentiment_label".

Khusus untuk kolom "clean_text_comment", "subjectivity", "polarity" dan "sentiment_label" adalah kolom yang dibuat saat melakukan proses text processing dan juga pelabelan dengan 3 nilai untuk mengklasifikasikan sentimen berdasarkan nilai/skor yang didapatkan pada kolom "polarity". Sedangkan kolom "subjectivity" merupakan kolom yang menunjukkan suatu kalimat komentar apakah bersifat subjektif atau objektif. Jika skor/nilai pada kalimat komentar memiliki nilai 0 maka dapat dipastikan jika komentar tersebut bersifat objektif sedangkan jika skor pada kalimat komentar memiliki nilai 1 maka komentar tersebut dapat dikategorikan sebagai komentar yang subjektif.

In [31]:
# Menampilkan tipe data yang dimiliki oleh masing-masing kolom dari variabel DataFrame "youtube_comments_df"
youtube_comments_df.dtypes

video_id                      object
channel_id                    object
channel_name                  object
comment_id                    object
author                        object
dirty_text_comment            object
clean_text_comment            object
like_count                     int64
total_reply                    int64
published_date        datetime64[ns]
updated_date          datetime64[ns]
subjectivity                 float64
polarity                     float64
sentiment_label               object
dtype: object

Berdasarkan hasil diatas dapat diketahui jika terdapat penambahan tipe data baru dari sebelumnya yaitu Float karena pada saat proses penggabungan pertama antara seluruh data komentar dari YouTube channel Najwa Shihab, KOMPASTV dan KPU RI, tidak memiliki tipe data Float. Namun tipe data Float yang saat ini dimiliki oleh kolom "subjectivity" dan "polarity" akan diubah menjadi tipe data Integer. Perlu diketahui dan diingat kembali jika tipe data Float merepresentasikan bilangan angka numerik desimal yaitu sebuah angka pecahan sedangkan tipe data Integer merepresentasikan bilangan bulat yaitu sebuah angka tanpa adanya floating point.

Selain itu, tipe data Object/String merupakan tipe data yang paling banyak dimiliki oleh kolom yang tersimpan di dalam variabel DataFrame "youtube_comments_df". Nama-nama kolom yang memiliki tipe data Object/String dari DataFrame "youtube_comments_df" adalah kolom "video_id", "channel_id", "channel_name", "comment_id", "author", "dirty_text_comment", "clean_text_comment", dan terakhir adalah kolom "sentiment_label". Sementara itu variabel DataFrame "youtube_comments_df" juga memiliki tipe data Datetime yang berfungsi sebagai representasi kolom yang menunjukkan stempel tanggal dan waktu seperti tanggal dan waktu pembelian suatu barang, tanggal dan waktu saat teks komentar dibuat oleh para penonton konten video di media sosial YouTube, dll.

In [32]:
"""
Menampilkan masing-masing kolom data dari variabel DataFrame
"youtube_comments_df" yang memiliki tipe data selain Object/String
"""
youtube_comments_df.select_dtypes(exclude="O")

,like_count,total_reply,published_date,updated_date,subjectivity,polarity
0,3970,332,2024-01-07 17:03:53,2024-01-09 23:18:49,0.300000,0.200000
1,3182,91,2023-12-12 21:22:08,2023-12-12 21:22:08,0.000000,0.000000
2,2936,124,2024-01-21 21:32:13,2024-01-21 21:32:13,0.000000,0.000000
3,2531,91,2024-01-22 00:17:18,2024-01-22 00:17:18,0.300000,0.200000
4,1590,122,2024-01-07 15:37:05,2024-01-07 15:37:05,0.474444,0.313333
...,...,...,...,...,...,...
48234,0,0,2024-01-07 19:08:29,2024-01-07 19:08:29,0.000000,0.000000
48235,0,0,2023-12-13 01:31:02,2023-12-13 01:31:02,0.000000,-0.100000
48236,0,0,2023-12-13 01:31:42,2023-12-13 01:31:42,0.900000,-0.800000
48237,0,0,2023-12-13 01:31:45,2023-12-13 01:31:45,0.000000,0.000000


In [33]:
"""
Menampilka kolom data dari variabel DataFrame
"youtube_comments_df" yang hanya memiliki tipe data datetime
"""
youtube_comments_df.select_dtypes(include="datetime")

,published_date,updated_date
0,2024-01-07 17:03:53,2024-01-09 23:18:49
1,2023-12-12 21:22:08,2023-12-12 21:22:08
2,2024-01-21 21:32:13,2024-01-21 21:32:13
3,2024-01-22 00:17:18,2024-01-22 00:17:18
4,2024-01-07 15:37:05,2024-01-07 15:37:05
...,...,...
48234,2024-01-07 19:08:29,2024-01-07 19:08:29
48235,2023-12-13 01:31:02,2023-12-13 01:31:02
48236,2023-12-13 01:31:42,2023-12-13 01:31:42
48237,2023-12-13 01:31:45,2023-12-13 01:31:45


In [13]:
# Menampilkan jumlah nilai data yang hilang dari masing-masing kolom dari variabel DataFrame "youtube_comments_df"
youtube_comments_df.isnull().sum()

video_id                 0
channel_id               0
channel_name             0
comment_id               0
author                   0
dirty_text_comment       0
clean_text_comment    1203
like_count               0
total_reply              0
published_date           0
updated_date             0
subjectivity             0
polarity                 0
sentiment_label          0
dtype: int64

In [19]:
youtube_comments_df_null = youtube_comments_df.isnull().any(axis=1)
null_rows = youtube_comments_df[youtube_comments_df_null]

null_rows.head()

,video_id,channel_id,channel_name,comment_id,author,dirty_text_comment,clean_text_comment,like_count,total_reply,published_date,updated_date,subjectivity,polarity,sentiment_label
2083,yNO0YS846kU,UCVdSz86o9q2cQKvAQ5QhOYg,KPU RI YouTube Channel,UgwAl-TcRXbfPGh-6d54AaABAg,@siapakah_saya,0,NaN,19,1,2023-12-12 15:02:44,2023-12-12 15:02:44,0.0,0.0,netral
2394,tEbRB5xxOPI,UCo8h2TY_uBkAVUIc14m_KCA,Najwa Shihab YouTube Channel,UgwcvlwzFm9WwBDcoFN4AaABAg,@FerryKur666,Makin Yakin Sama Pak 03 ❤,NaN,15,0,2024-01-21 18:07:34,2024-01-21 18:07:34,0.0,0.0,netral
2428,8J66JxvmEzo,UCVdSz86o9q2cQKvAQ5QhOYg,KPU RI YouTube Channel,UgwzkqWxAeXqc3agLVJ4AaABAg,@L.Am.AnRony1002,✌️✌️,NaN,15,0,2024-02-04 15:21:52,2024-02-04 15:21:52,0.0,0.0,netral
2916,8J66JxvmEzo,UCVdSz86o9q2cQKvAQ5QhOYg,KPU RI YouTube Channel,UgwDg_vJ4gldo1gy_HR4AaABAg,@theleggend925,✌️✌️✌️✌️✌️💯💯💯💯,NaN,10,0,2024-02-04 16:01:16,2024-02-04 16:01:16,0.0,0.0,netral
2939,8J66JxvmEzo,UCVdSz86o9q2cQKvAQ5QhOYg,KPU RI YouTube Channel,UgzVsY1W2uZTinSCdFt4AaABAg,@dewiekaprawitarani3700,❤1❤1❤1,NaN,10,0,2024-02-04 15:20:22,2024-02-04 15:20:22,0.0,0.0,netral
